# Example of preprocessing pipeline 

### `Prerequisites:`

1. #### Acquire data and manage them according to the [data management SOP]()

2. #### In case you haven't installed `gks2p` package, install it from [here](https://github.com/gkeliris/gks2p_preprocess).

3. #### Make sure you have prepared an `[Experiment].csv` file, see example [here](/home/georgioskeliris/GitHub/gks2p_preprocess/scripts/LRN2P_datasets.csv)



### First, import the `gks2p` package functions as follows:

In [2]:
from gks2p.datasets import *
from gks2p.preprocess import *
from gks2p.mkops import *

### Then, set the `basepath` and `fastbase` paths.
 **basepath** is the base location (usually on the NAS) where the processed data is stored (see [data management SOP]()).

 **fastbase** can be the same location or a local SSD for fast processing. This is where the binary suite2p files are stored. In our pipeline these binary files can be used again to import data for other analyses and thus they shouldn't be immediately deleted. Thus having that on the same location on the NAS is more convenient but could be a bit slower. Alternatively, an SSD can be used and they can be copied to the NAS later. However, in that case some other files may also need to be updated to have the correct paths to the binary data.

In [3]:
basepath = '/mnt/NAS_DataStorage/Data_active/LRN2P/'
fastbase = '/mnt/NAS_DataStorage/Data_active/LRN2P/'

### The following python dictionary (`db`) is used to change some suite2p parameters from their default values

*Note that first the default values are created and then the value pairs in db are overwriting those.*

The `pipeline` parameter is used as a way to be able to run the analysis again with different parameters without overwriting the previous analysis (e.g. for comparisons). It defaults to the value `orig`. You can change it to some other name for a second analysis etc.

The `tau` is the time-constant of the calcium sensitive dye (CSD) and used for the deconvolution. However, it seems to be also used for some smooting operations and in my experience the value of `1.5`, which is the time-constant of the slow variants of GCaMP6, seems to perform better for segmentation of some of our data - even if the fast GCaMP variant was used. This pipeline, allows to use this default value for the segmentation step, and then change it later to perform the deconvolution with the one appropriate with the correct tau of the used GCaMP variant.

A `spatial scale` of 0, indicates to suite2p to try and automatically identify the spatial scale. This has to do with the size of the cells in pixels and it is usually around 6 (but it depends of course on the imaging resolution). If you know the value you want to use you can simply enter that instead of 0.

The `reg_tif` argument indicates to suite2p whether to store the registered (motion corrected) tifs again. This is useful for exampel to be able to use these images in different softwhere. For the needs of our pipeline this is not necesssary because we have routines to directly import the binary data (note that the motion corrected binary are also stored by suite2p). Thus, to avoid over-duplication of the data we keep that to `False` unless needed for something else.

For other parameters that can be added in the dictionary please check the [suite2p documentation](https://suite2p.readthedocs.io/en/latest/settings.html)

In [4]:
db = {
    'pipeline': 'orig',
    'tau': 1.5,
    'spatial_scale': 0,
    'reg_tif': False
}

### Use the `datasetQuery` function to select datasets based on parameters such as *animal id*, *exp id*, *etc*.

This will retrieve the matching dataset(s) from the [Experiment].csv files

**MULTIDATA PROCESSING NOTE:**
if **ds** has multiple datasets, the commands below will iterate and process all datasets one after another. However, for high memory consuming tasks, such as motion correction or segmentation it is probably best to not overload this with too many datasets at the same time. I noticed that such processes probably fail to clear their used resources completely, as the more you try to run the slower it gets.

In [5]:

ds = datasetQuery(cohort='lrn1', day='d0', experiment='DR')

### Read the header information from the .tif files and create `ops files`

`ops files` are files that can be used as input to suite2p and define different parameters of the data (such as sampling rate, number of planes etc.) plus the parameters that control the suite2p algorithms

In [7]:
gks2p_makeOps(ds, basepath, db=db, fastbase=fastbase)




PROCESSING:
datID                                                        1
cohort                                                    lrn1
day                                                         d0
session                                                   ses1
mouseID                                                   M827
expID                                                       DR
rawPath      /mnt/NAS_UserStorage/Mingyu/learning/2P imagin...
firstTiff            M827_thy1_tg1_20250121_DR_00001_00001.tif
matfolder    /mnt/NAS_UserStorage/Mingyu/Eyetracking/learni...
matfile            M827_thy1tg1_DR_21_Jan_2025_15_35_45_01.mat
eyefolder                                                  NaN
eyefile                                                    NaN
Name: 0, dtype: object
[579, 290, 0]
[0, 0, 0]


### Convert the .tif files to suite2p binary format and store

The suite2p binaries are essentially matrices per scan ROI (thus notice that a multiplane scan will be converted to several files - a folder per plane)

In [8]:
gks2p_toBinary(ds, basepath)

datID                                                        1
cohort                                                    lrn1
day                                                         d0
session                                                   ses1
mouseID                                                   M827
expID                                                       DR
rawPath      /mnt/NAS_UserStorage/Mingyu/learning/2P imagin...
firstTiff            M827_thy1_tg1_20250121_DR_00001_00001.tif
matfolder    /mnt/NAS_UserStorage/Mingyu/Eyetracking/learni...
matfile            M827_thy1tg1_DR_21_Jan_2025_15_35_45_01.mat
eyefolder                                                  NaN
eyefile                                                    NaN
Name: 0, dtype: object
{}
NOTE: nplanes 1 nrois 3 => ops['nplanes'] = 3
mesoscan
** Found 21 tifs - converting to binary **
6000 frames of binary, time 100.22 sec.
12000 frames of binary, time 188.99 sec.
time 200.71 sec. Wrote 12600 frames per

### Motion correction (i.e. registration)

Note that by default this will register all the planes of a particular dataset (when **iplaneList=None**). If sometimes the computer fails after a particular plane or for some reason you would like to (re)register particular planes the iplaneList argument can be used and given the specific planes (e.g. **iplaneList=[0 3 8]**)

`Note:` *The motion corrected data are also stored as suite2p binary files in the same folder with the raw binary files*

In [9]:
gks2p_register(ds, basepath, iplaneList=None)

datID                                                        1
cohort                                                    lrn1
day                                                         d0
session                                                   ses1
mouseID                                                   M827
expID                                                       DR
rawPath      /mnt/NAS_UserStorage/Mingyu/learning/2P imagin...
firstTiff            M827_thy1_tg1_20250121_DR_00001_00001.tif
matfolder    /mnt/NAS_UserStorage/Mingyu/Eyetracking/learni...
matfile            M827_thy1tg1_DR_21_Jan_2025_15_35_45_01.mat
eyefolder                                                  NaN
eyefile                                                    NaN
Name: 0, dtype: object

REGISTERING: plane0
Reference frame, 72.33 sec.
Registered 500/12600 in 40.77s
Registered 1000/12600 in 82.09s
Registered 1500/12600 in 100.19s
Registered 2000/12600 in 129.68s
Registered 2500/12600 in 171.02s
Registered

### Segmentation of the images to ROIs (cells / non-cells)

> This is the most critical step. One advantage of having the suite2p processes broken down to sub-modules is that one can repeat some critical steps with adjusted parameters until the result is satisfactory. If for example the result is problematic one can adjust some parameters in **db** and run again **gks2p_makeOps** before trying again **gks2p_segment**. Importantly, parameters are not important for **gks2p_toBinary** and usually also not important for **gks2p_register** (thus those two very time consuming steps can be skipped saving much time)

In [10]:
gks2p_segment(ds, basepath, iplaneList=None)

datID                                                        1
cohort                                                    lrn1
day                                                         d0
session                                                   ses1
mouseID                                                   M827
expID                                                       DR
rawPath      /mnt/NAS_UserStorage/Mingyu/learning/2P imagin...
firstTiff            M827_thy1_tg1_20250121_DR_00001_00001.tif
matfolder    /mnt/NAS_UserStorage/Mingyu/Eyetracking/learni...
matfile            M827_thy1tg1_DR_21_Jan_2025_15_35_45_01.mat
eyefolder                                                  NaN
eyefile                                                    NaN
Name: 0, dtype: object

SEGMENTING: plane0
Binning movie in chunks of length 09
Binned movie of size [1400,1302,294] created in 15.17 sec.
NOTE: estimated spatial scale ~6 pixels, time epochs 1.17, threshold 5.83 
0 ROIs, score=255.37
1000 ROIs

/home/georgioskeliris/GitHub/suite2p/suite2p/extraction/extract.py:125: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  Fi[n] = np.dot(data[:, cell_ipix[n]], cell_lam[n])


Extracted fluorescence from 4824 ROIs in 12600 frames, 39.52 sec.
['npix_norm', 'skew', 'compact']

SEGMENTING: plane1
Binning movie in chunks of length 09
Binned movie of size [1400,1298,292] created in 15.11 sec.
NOTE: estimated spatial scale ~6 pixels, time epochs 1.17, threshold 5.83 
0 ROIs, score=184.16
1000 ROIs, score=50.58
2000 ROIs, score=32.51
3000 ROIs, score=23.69
4000 ROIs, score=18.12
Detected 5000 ROIs, 127.06 sec
After removing overlaps, 4866 ROIs remain
Masks created, 8.25 sec.
Extracted fluorescence from 4866 ROIs in 12600 frames, 34.14 sec.
['npix_norm', 'skew', 'compact']

SEGMENTING: plane2
Binning movie in chunks of length 09
Binned movie of size [1400,1296,288] created in 15.06 sec.
NOTE: estimated spatial scale ~6 pixels, time epochs 1.17, threshold 5.83 
0 ROIs, score=183.41
1000 ROIs, score=43.21
2000 ROIs, score=25.36
3000 ROIs, score=16.91
4000 ROIs, score=12.02
Detected 5000 ROIs, 107.80 sec
After removing overlaps, 4676 ROIs remain
Masks created, 8.28 sec

### (Re)classigy cell / non-cells using another (onw-trained) classifier

Sometimes the suite2p native classifer that classifies ROIs to cells and non-cells based on a selected probability doesn't perform optimally for different types of datasets. However, suite2p allows some manual correction of these categories (curation) and in addition allows a training of a classifier based on these curated data. Thus, if you have already curated multiple datasets and create a new classifier, there is a much better chance that this will perform better in newly acquired data of the same type and will thus not require too much effort for curation.

In [ ]:
gks2p_classify(ds, basepath, classfile=[paht_to_classifier])

### Deconvolution using a different tau than segmentation

In [ ]:
gks2p_deconvolve(ds,basepath,0.7) # 0.7 here is the tau value

### Combine the single plane results to a single plane 

#### The `gks2p_combine(ds, basepath)` command combines the results from multiple planes or ROIs (Regions of Interest) after segmentation and processing steps.

#### It aggregates the outputs (such as extracted signals, masks, and metadata) into unified files or structures for downstream analysis.

#### This step is typically performed after motion correction and segmentation to facilitate further data analysis and visualization.

`Note:` *This is mostly relevant for viewing all simultaneously in suite2p but in fact for other purposes like careful stitching and alignment, we use other procedures*

In [12]:
gks2p_combine(ds, basepath)

[300 300 300]
879.0
1308.0
[579 290   0]
[0 0 0]
appended plane 0 to combined view
appended plane 1 to combined view
appended plane 2 to combined view


## How to import in the pipeline datasets for which suite2p has already been runned

In [ ]:
gks2p_import(dat, import_folder, basepath, fastbase=None, db={})

## Use `FISSA` to remove the effects of the neuropil and calculate DeltaF/F0

#### `Prerequisites:`
1. Install FISSA

**Quick Guide:**
>   pip install fissa

For more info check this [link](https://github.com/rochefort-lab/fissa)

In [ ]:
# We assume that the gks2p package modules have already been imported, else we would need to import them here.

# Here we import FISSA for further processing.
import fissa

#### Process a dataset with `FISSA` to remove neuropil contamination from the cell ROIs and calculate the deltaF/F0

In [ ]:
# Load the dataset of interest using the datasetQuery function.
ds = datasetQuery(cohort='lrn1', day='d6', mouseID='M827', experiment='FamNov2')

gks2p_fissa(ds, basepath, iplaneList=None, nCores=None, use_reg_tif=False, redo_prep=False)
